<a href="https://colab.research.google.com/github/carvalheiracarlos/deep_leaning_notebooks/blob/main/alzheimer_cnn/testdrive_lenet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

In [4]:
from keras.callbacks import ModelCheckpoint
from keras.metrics import categorical_accuracy
from keras.models import Sequential
from keras.layers.convolutional import Convolution2D
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras import backend
import tensorflow as tf
import json

In [10]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [2]:
X_train = np.load('drive/MyDrive/alzheimer_datasets/X_train.npy', allow_pickle=True)
X_test = np.load('drive/MyDrive/alzheimer_datasets/X_test.npy', allow_pickle=True)

y_train = np.load('drive/MyDrive/alzheimer_datasets/y_train.npy', allow_pickle=True)
y_test = np.load('drive/MyDrive/alzheimer_datasets/y_test.npy', allow_pickle=True)

In [7]:
X_train[0].shape

(190, 200, 1)

In [5]:
class LeNet:
  @staticmethod
  def build(n_channels, 
            rows, 
            cols, 
            n_classes, 
            activation='relu', 
            weights_path=None):
    model = Sequential()
    
    input_Shape = (rows, cols, n_channels)
    
   
    #Convolution and Pooling Layers
    model.add(Conv2D(20, kernel_size=5, activation=activation, input_shape=input_Shape))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    
    model.add(Conv2D(50, kernel_size=5, activation=activation))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    
    #Flatten and Fully Conected layer
    model.add(Flatten())
    model.add(Dense(32))
    model.add(Activation(activation))
    
    # OutputLayer SOFTMAX activation
    model.add(Dense(n_classes))
    model.add(Activation('softmax'))
    
    # if a weights path is supplied (inicating that the model was
    # pre-trained), then load the weights
    if weights_path is not None:
      model.load_weights(weights_path)
    
    return model

In [9]:
n_epochs = 10
ini_learning_rate = 1e-3
batch_size = 32

In [11]:
with tf.device('/device:GPU:0') as sess:
    train_model = LeNet.build(n_channels=1, 
                              rows=190,
                              cols=200, 
                              n_classes=26, 
                              activation='relu', 
                              weights_path=None)

    opt = tf.keras.optimizers.Adam(lr=ini_learning_rate, decay=ini_learning_rate / n_epochs)
    train_model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=[categorical_accuracy])
    try:
        file_path = 'weights/alpha_weights_Letras.h5'
        check_point = ModelCheckpoint(file_path, 
                                      monitor='categorical_accuracy', 
                                      verbose=0, 
                                      save_best_only=True,
                                      mode='max',
                                      save_weights_only=True)
        callbacks_list = [check_point]
        
        train_history = train_model.fit(x=X_train, 
                                        y=y_train, 
                                        batch_size=batch_size, 
                                        epochs=n_epochs,
                                        callbacks=callbacks_list,
                                        verbose=1)
    
   
    except IOError:
        print('Error while saving the Model weights')



/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


ValueError: ignored